# IPython Blockfolio

This is a simple "Blockfolio" using Python and the [CoinMarketCap API](https://coinmarketcap.com/api/).

It will show the net worth in USD of your Cryptocurrency investments as well as the relative share
of coins you own in comparison to the total coin supply of the corresponding digital asset.

First, the current exchanges rates are loaded, then combined with your investments, and finally displayed using Plotly.

This notebook can be found on [my github profile](https://github.com/SmokinCaterpillar/blockfolio).

Imports
--------

In [1]:
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

## Loading Data from CoinMarkteCap

In [2]:
limit = 200  # only load the top 200 currencies, if you have invested in smaller ones increase this limit
coin_market_cap_api = 'https://api.coinmarketcap.com/v1/ticker/?limit={}'.format(limit)
coin_market_cap_api

'https://api.coinmarketcap.com/v1/ticker/?limit=200'

In [3]:
# load the data using pandas
market_data = pd.read_json(coin_market_cap_api)
market_data.head(10)

,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,symbol,total_supply
0,1.127430e+10,16769287,bitcoin,1514540960,245668377621,2.100000e+07,Bitcoin,-1.56,0.95,7.11,1.000000,14649.900000,1,BTC,16769287
1,2.406740e+09,96638893,ethereum,1514540949,71605071078,NaN,Ethereum,-1.72,3.09,10.00,0.051113,740.955000,2,ETH,96638893
2,3.511390e+09,38739144847,ripple,1514541241,65368045623,1.000000e+11,Ripple,4.44,26.20,58.78,0.000117,1.687390,3,XRP,99993093880
3,3.541710e+09,16881938,bitcoin-cash,1514540953,41801871902,2.100000e+07,Bitcoin Cash,-2.15,-5.81,-0.37,0.170809,2476.130000,4,BCH,16881938
4,3.020040e+09,54521983,litecoin,1514540941,13322337540,8.400000e+07,Litecoin,-1.89,-3.66,-3.04,0.016856,244.348000,5,LTC,54521983
5,1.056140e+08,25927070538,cardano,1514540954,11408455505,4.500000e+10,Cardano,1.47,14.56,12.98,0.000030,0.440021,6,ADA,31112483745
6,1.922570e+08,2779530283,iota,1514540952,9960307793,2.779530e+09,IOTA,-3.53,3.40,-3.47,0.000247,3.583450,7,MIOTA,2779530283
7,1.607890e+08,7781857,dash,1514541242,8576929252,1.890000e+07,Dash,-1.89,0.65,-0.36,0.076292,1102.170000,8,DASH,7781857
8,4.756490e+07,8999999999,nem,1514540944,8213129999,NaN,NEM,-1.49,4.46,12.24,0.000063,0.912570,9,XEM,8999999999
9,1.365450e+08,15529227,monero,1514541243,5700980980,NaN,Monero,-1.18,-1.39,13.16,0.025412,367.113000,10,XMR,15529227


## Your Blockfolio

Below you can fill in the types and amount of coins you have into the Python dictionary
with the format `'SYMBOL': value`.

In [4]:
# enter your coins below:

block_folio = {
    'BTC': 0.31337,
    'BCH': 0.01337,
    'ETH': 3,
    'LTC': 1.5,
    'MIOTA': 777,
    'XRP': 13,
    'DASH': 0.1, 
    'XMR': 1, 
    'LSK': 12, 
    'OMG': 1.1, 
    'XRB': 42, 
    'PIVX': 123, 
    'ARK': 22, 
    'NEO': 7, 
    'GAS': 0.01,
    'STEEM': 499,
}

# display your blockfolio
block_folio = pd.DataFrame(list(block_folio.items()), columns=['symbol', 'amount'])
block_folio.head(len(block_folio))

,symbol,amount
0,ETH,3.00000
1,STEEM,499.00000
2,DASH,0.10000
3,GAS,0.01000
4,OMG,1.10000
5,LSK,12.00000
6,XRB,42.00000
7,MIOTA,777.00000
8,NEO,7.00000
9,ARK,22.00000


### Combining the Blockfolio and the API Data

In [5]:
# merge the API and blockfolio data and sort by investment value
merged_data = block_folio.merge(market_data, how='left')
merged_data['value_usd'] = merged_data.amount * merged_data.price_usd
merged_data['coinshare'] = merged_data.amount / merged_data.available_supply
merged_data = merged_data.sort_values('value_usd', ascending=False)
merged_data.head()

,symbol,amount,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,total_supply,value_usd,coinshare
13,BTC,0.31337,1.127430e+10,16769287,bitcoin,1514540960,245668377621,2.100000e+07,Bitcoin,-1.56,0.95,7.11,1.000000,14649.90000,1,16769287,4590.839163,1.868714e-08
7,MIOTA,777.00000,1.922570e+08,2779530283,iota,1514540952,9960307793,2.779530e+09,IOTA,-3.53,3.40,-3.47,0.000247,3.58345,7,2779530283,2784.340650,2.795436e-07
0,ETH,3.00000,2.406740e+09,96638893,ethereum,1514540949,71605071078,NaN,Ethereum,-1.72,3.09,10.00,0.051113,740.95500,2,96638893,2222.865000,3.104340e-08
1,STEEM,499.00000,4.600670e+06,246343141,steem,1514540946,704602968,NaN,Steem,-1.25,-0.01,5.94,0.000197,2.86025,41,263317235,1427.264750,2.025630e-06
14,PIVX,123.00000,7.361070e+06,55233482,pivx,1514540946,617819641,NaN,PIVX,0.65,-2.51,42.95,0.000772,11.18560,44,55233482,1375.828800,2.226910e-06


### Your Blockfolio in numbers and graphs!


In [6]:
networth = 'Your blockfolio is currently worth {:.2f} USD!'.format(merged_data.value_usd.sum())
print(networth)

Your blockfolio is currently worth 14831.89 USD!


#### Net Worth

Let's plot the worth of each asset in descending order:

In [7]:
marker = dict(color='rgb(158,202,225)',
              line=dict(color='rgb(8,48,107)', width=1.5))

layout = go.Layout(title='Blockfolio Networth in USD',
                   xaxis=dict(title='Cryptocurrency',),
                   yaxis=dict(title='USD'))
              
value_chart=go.Bar(x=merged_data.symbol, y=merged_data.value_usd, marker=marker)

fig = go.Figure(data=[value_chart], layout=layout)
py.iplot(fig)

#### Fraction of total Supply
Finally, that looks at the relative sizes of your investment. 
This gives you an idea or visualizes in which cryptocurrency you believe in the most. 
The graph below shows you fractional share of the the current crypto's total supply.

In [8]:
marker = dict(color='rgb(258,202,125)', line=dict(color='rgb(8,48,107)', width=1.5,))

layout = go.Layout(title='Relative Blockfolio Size',
                   xaxis=dict(title='Cryptocurrency',),
                   yaxis=dict(title='Fraction of total Supply'))

share_chart=go.Bar(x=merged_data.symbol, y=merged_data.coinshare, marker=marker)

fig = go.Figure(data=[share_chart], layout=layout)
py.iplot(fig)